In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,71.65,78,100,11.50,overcast clouds
1,1,Jamestown,US,42.0970,-79.2353,77.02,59,100,9.22,overcast clouds
2,2,Mar Del Plata,AR,-38.0023,-57.5575,55.09,98,100,3.00,overcast clouds
3,3,Boa Vista,BR,2.8197,-60.6733,80.26,79,99,2.10,overcast clouds
4,4,Albany,US,42.6001,-73.9662,80.37,51,99,0.65,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria.. 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 90


In [4]:
# 3. Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 
                                    
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,71.65,78,100,11.50,overcast clouds
1,1,Jamestown,US,42.0970,-79.2353,77.02,59,100,9.22,overcast clouds
3,3,Boa Vista,BR,2.8197,-60.6733,80.26,79,99,2.10,overcast clouds
4,4,Albany,US,42.6001,-73.9662,80.37,51,99,0.65,overcast clouds
7,7,Rikitea,PF,-23.1203,-134.9692,74.10,73,98,12.73,overcast clouds
12,12,Eberbach,DE,49.4694,8.9928,75.54,45,85,1.01,overcast clouds
13,13,Linhares,BR,-19.3911,-40.0722,80.01,43,4,9.91,clear sky
16,16,Seoni Malwa,IN,22.4500,77.4667,76.42,95,86,6.51,overcast clouds
20,20,Ajdabiya,LY,30.7554,20.2263,81.28,67,1,16.28,clear sky
21,21,Atuona,PF,-9.8000,-139.0333,77.29,67,33,20.83,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                331
City                   331
Country                331
Lat                    331
Lng                    331
Max Temp               331
Humidity               331
Cloudiness             331
Wind Speed             331
Current Description    331
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,71.65,78,100,11.50,overcast clouds
1,1,Jamestown,US,42.0970,-79.2353,77.02,59,100,9.22,overcast clouds
3,3,Boa Vista,BR,2.8197,-60.6733,80.26,79,99,2.10,overcast clouds
4,4,Albany,US,42.6001,-73.9662,80.37,51,99,0.65,overcast clouds
7,7,Rikitea,PF,-23.1203,-134.9692,74.10,73,98,12.73,overcast clouds


In [7]:
clean_travel_cities.count()

City_ID                331
City                   331
Country                331
Lat                    331
Lng                    331
Max Temp               331
Humidity               331
Cloudiness             331
Wind Speed             331
Current Description    331
dtype: int64

In [8]:
clean_travel_cities.columns

Index(['City_ID', 'City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Current Description'],
      dtype='object')

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,71.65,overcast clouds,-21.2078,-159.7750,
1,Jamestown,US,77.02,overcast clouds,42.0970,-79.2353,
3,Boa Vista,BR,80.26,overcast clouds,2.8197,-60.6733,
4,Albany,US,80.37,overcast clouds,42.6001,-73.9662,
7,Rikitea,PF,74.10,overcast clouds,-23.1203,-134.9692,
12,Eberbach,DE,75.54,overcast clouds,49.4694,8.9928,
13,Linhares,BR,80.01,clear sky,-19.3911,-40.0722,
16,Seoni Malwa,IN,76.42,overcast clouds,22.4500,77.4667,
20,Ajdabiya,LY,81.28,clear sky,30.7554,20.2263,
21,Atuona,PF,77.29,scattered clouds,-9.8000,-139.0333,


In [10]:
# 6a. Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,71.65,overcast clouds,-21.2078,-159.7750,Paradise Inn
1,Jamestown,US,77.02,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
3,Boa Vista,BR,80.26,overcast clouds,2.8197,-60.6733,Aipana Plaza Hotel
4,Albany,US,80.37,overcast clouds,42.6001,-73.9662,
7,Rikitea,PF,74.10,overcast clouds,-23.1203,-134.9692,People ThankYou
12,Eberbach,DE,75.54,overcast clouds,49.4694,8.9928,Altes Badhaus
13,Linhares,BR,80.01,clear sky,-19.3911,-40.0722,Hotel Conceição
16,Seoni Malwa,IN,76.42,overcast clouds,22.4500,77.4667,hotel naman residency
20,Ajdabiya,LY,81.28,clear sky,30.7554,20.2263,Almadina Hotel
21,Atuona,PF,77.29,scattered clouds,-9.8000,-139.0333,Villa Enata


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,71.65,overcast clouds,-21.2078,-159.7750,Paradise Inn
1,Jamestown,US,77.02,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
3,Boa Vista,BR,80.26,overcast clouds,2.8197,-60.6733,Aipana Plaza Hotel
7,Rikitea,PF,74.10,overcast clouds,-23.1203,-134.9692,People ThankYou
12,Eberbach,DE,75.54,overcast clouds,49.4694,8.9928,Altes Badhaus
13,Linhares,BR,80.01,clear sky,-19.3911,-40.0722,Hotel Conceição
16,Seoni Malwa,IN,76.42,overcast clouds,22.4500,77.4667,hotel naman residency
20,Ajdabiya,LY,81.28,clear sky,30.7554,20.2263,Almadina Hotel
21,Atuona,PF,77.29,scattered clouds,-9.8000,-139.0333,Villa Enata
23,Aksay,RU,77.25,broken clouds,47.2600,39.8700,Chernaya Zhemchuzhina


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))